In [3]:
####### (a) #######

# This code is used for JHU CS 482/682: Deep Learning 2019 Spring Homework 4
# Copyright @ Johns Hopkins University, Cong Gao, cgao11@jhu.edu
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import FashionMNIST

# IMPORTANT: This function is used for de-normalizing image to original domain,
# please use this if you want to visualize/recover your output result
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

# These hyperparameters can be changed
num_epochs = 10
batch_size = 128
learning_rate = 1e-3

# This is a recommended normalization method. You can design your own
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
])

# Dataset wiill be downloaded to './data' folder
train_dataset = FashionMNIST('./data', train=True, transform=img_transform, download=True)
test_dataset = FashionMNIST('./data', train=False, transform=img_transform, download=True)
train_loader = DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True);
test_loader = DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True);

In [4]:
######### (a)-continued #######
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.ly1 = nn.Sequential(
            nn.Conv2d(1, 4, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.ly2 = nn.Sequential(
            nn.Conv2d(4, 8, 4, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.ly3 = nn.Sequential(
            nn.Conv2d(8, 16, 5, 1, 3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.linear = nn.Linear(256,10)
        
    def forward(self, x):
        x = self.ly1(x)
        x = self.ly2(x)
        x = self.ly3(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

In [5]:
######### (b) #########
### Training
cnn = CNN();

criterion = nn.CrossEntropyLoss();
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate, weight_decay=1e-5);

losses = [];
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.float())
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.data.item());
        
    print("Epoch: %d" % (epoch+1))
    print("Training Loss: %.4f" % (loss.data.item()))
    
### Testing
cnn.eval()
test_correct = 0
test_num = 0
for (images, labels) in test_loader:
    images = Variable(images.float())
    outputs = cnn(images)
    a, predicted = torch.max(outputs.data, 1)
    test_num += len(labels)
    test_correct += (predicted == labels).sum().item()
print("Test Accuracy is: " + str(test_correct/test_num*100) + "%")

Epoch: 1
Training Loss: 0.5979
Epoch: 2
Training Loss: 0.3705
Epoch: 3
Training Loss: 0.4828
Epoch: 4
Training Loss: 0.4454
Epoch: 5
Training Loss: 0.3996
Epoch: 6
Training Loss: 0.4554
Epoch: 7
Training Loss: 0.3040
Epoch: 8
Training Loss: 0.3436
Epoch: 9
Training Loss: 0.3631
Epoch: 10
Training Loss: 0.3579
Test Accuracy is: 88.02%


In [0]:
####### (c) #######
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.ly1 = nn.Sequential(
            nn.Conv2d(1, 4, 3, 1, 1),
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.ly2 = nn.Sequential(
            nn.Conv2d(4, 8, 4, 1, 2),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.ly3 = nn.Sequential(
            nn.Conv2d(8, 16, 5, 1, 3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2)
        )
        self.linear = nn.Linear(256,10)
        
    def forward(self, x):
        x = self.ly1(x)
        x = self.ly2(x)
        x = self.ly3(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

In [0]:
####### (c)-continued #######
### Training with the improved model
cnn2 = CNN2();

criterion = nn.CrossEntropyLoss();
optimizer = torch.optim.Adam(cnn2.parameters(), lr=learning_rate);

losses = [];
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.float())
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn2(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.data.item());
        
    print("Epoch: %d" % (epoch+1))
    print("Training Loss: %.4f" % (loss.data.item()))

### Testing with the improved model
cnn2.eval()
test_correct = 0
test_num = 0
for (images, labels) in test_loader:
    images = Variable(images.float())
    outputs = cnn2(images)
    a, predicted = torch.max(outputs.data, 1)
    test_num += len(labels)
    test_correct += (predicted == labels).sum().item()
print("Test Accuracy is: " + str(test_correct/test_num*100) + "%")

Epoch: 1
Training Loss: 0.5951
Epoch: 2
Training Loss: 0.3648
Epoch: 3
Training Loss: 0.2870
Epoch: 4
Training Loss: 0.2950
Epoch: 5
Training Loss: 0.3155
Epoch: 6
Training Loss: 0.4100
Epoch: 7
Training Loss: 0.1763
Epoch: 8
Training Loss: 0.2107
Epoch: 9
Training Loss: 0.2153
Epoch: 10
Training Loss: 0.2289
Test Accuracy is: 89.69%
